In [ ]:
import os
import shutil

# Example code to organize the dataset into train and test folders
src_dir = '/path/to/extracted/dataset'
dst_dir = '/path/to/dataset_split'

train_ratio = 0.8  # Adjust the train-test split ratio

for class_name in os.listdir(src_dir):
    class_dir = os.path.join(src_dir, class_name)
    files = os.listdir(class_dir)
    num_train = int(len(files) * train_ratio)

    # Create train and test directories
    train_dir = os.path.join(dst_dir, 'train', class_name)
    test_dir = os.path.join(dst_dir, 'test', class_name)
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Move files to train and test directories
    for i, file in enumerate(files):
        src_path = os.path.join(class_dir, file)
        if i < num_train:
            dst_path = os.path.join(train_dir, file)
        else:
            dst_path = os.path.join(test_dir, file)
        shutil.move(src_path, dst_path)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    '/path/to/dataset_split/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    '/path/to/dataset_split/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Load the pre-trained VGG16 model without the top (classification) layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of the base model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_generator, epochs=10, validation_data=test_generator)

In [ ]:
calorie_model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='linear')  # Regression output for calorie estimation.
])

# Compile the calorie estimation model
calorie_model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the calorie estimation model using a dataset with images and calorie values
calorie_model.fit(calorie_train_generator, epochs=10)

In [ ]:
calorie_loss = calorie_model.evaluate(calorie_test_data, calorie_test_labels)
print(f'Calorie Estimation Loss: {calorie_loss}')

In [ ]:
# Predict the food item category using the classification model
predicted_class = model.predict(user_provided_image)

# Predict the calorie content using the calorie estimation model
predicted_calories = calorie_model.predict(user_provided_image)